In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder


In [14]:
class ANN:
    def __init__(self, input_size, hidden_size_1, hidden_size_2, output_size, learning_rate):
        self.input_size = input_size
        self.hidden_size_1 = hidden_size_1
        self.hidden_size_2 = hidden_size_2
        self.output_size = output_size
        self.learning_rate = learning_rate
        
        # Initialize weights and biases
        self.Wij = np.random.uniform(-0.5, 0.5, (self.input_size, self.hidden_size_1))
        self.Vjk = np.random.uniform(-0.5, 0.5, (self.hidden_size_1, self.hidden_size_2))
        self.Ukl = np.random.uniform(-0.5, 0.5, (self.hidden_size_2, self.output_size))
        
        self.bias_j = np.random.uniform(-0.5, 0.5, (1, self.hidden_size_1)) 
        self.bias_k = np.random.uniform(-0.5, 0.5, (1, self.hidden_size_2))
        self.bias_l = np.random.uniform(-0.5, 0.5, (1, self.output_size))
        
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
     
    def sigmoid_derivative(self, x):
        return x * (1 - x)
    
    def forward_prop(self, x):
        self.hidden_layer_1_input = np.dot(x, self.Wij) + self.bias_j 
        self.hidden_layer_1_output = self.sigmoid(self.hidden_layer_1_input)
        
        self.hidden_layer_2_input = np.dot(self.hidden_layer_1_output, self.Vjk) + self.bias_k
        self.hidden_layer_2_output = self.sigmoid(self.hidden_layer_2_input)
        
        self.output_layer_input = np.dot(self.hidden_layer_2_output, self.Ukl) + self.bias_l
        self.output_layer_output = self.sigmoid(self.output_layer_input)
        return self.output_layer_output
    
    def backprop(self, x, y, output):
        error = y - output
        delta_output = error * self.sigmoid_derivative(output)
        
        hidden_layer_2_error = delta_output.dot(self.Ukl.T)
        hidden_layer_2_delta = hidden_layer_2_error * self.sigmoid_derivative(self.hidden_layer_2_output)
        
        hidden_layer_1_error = hidden_layer_2_delta.dot(self.Vjk.T)
        hidden_layer_1_delta = hidden_layer_1_error * self.sigmoid_derivative(self.hidden_layer_1_output)
        

        self.Ukl += self.hidden_layer_2_output.T.dot(delta_output) * self.learning_rate
        self.bias_l += np.sum(delta_output) * self.learning_rate
        
        self.Vjk += self.hidden_layer_1_output.T.dot(hidden_layer_2_delta) * self.learning_rate
        self.bias_k += np.sum(hidden_layer_2_delta) * self.learning_rate
         
        self.Wij += x.T.dot(hidden_layer_1_delta) * self.learning_rate
        self.bias_j += np.sum(hidden_layer_1_delta) * self.learning_rate
        
    def train(self, x, y, epochs):
        for epoch in range(epochs):
            output = self.forward_prop(x)
            self.backprop(x, y, output)
            if epoch % 1000 == 0:
                print(f'Epoch {epoch}: loss {np.mean(np.abs(y - output))}')

In [15]:
df = pd.read_csv("Iris.csv")

In [16]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [17]:
df["Species"].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [18]:
df["Species"] = df["Species"].replace({'Iris-setosa':0, 'Iris-versicolor':1, 'Iris-virginica':2})
df["Species"].unique()

array([0, 1, 2], dtype=int64)

In [19]:
arr1 = np.array([df['SepalLengthCm'], df['SepalWidthCm'], df['PetalLengthCm'], df['PetalWidthCm']]).T
arr2 = np.array([df['Species']]).T

arr1,arr2

(array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
        [5

In [24]:
x_train, x_test, y_train, y_test = train_test_split(arr1, arr2, test_size=0.2, random_state=10)

In [25]:
encoder = OneHotEncoder(categories='auto')

In [30]:
y_train_hot = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
y_train_hot

array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1

In [27]:
input_size = x_train.shape[1]
hidden_size_1 = 50
hidden_size_2 = 50
output_size = y_train_hot.shape[1]
learning_rate = 0.1
epochs = 10000

In [28]:
obj_ANN = ANN(input_size, hidden_size_1, hidden_size_2, output_size, learning_rate)
obj_ANN.train(x_train, y_train_hot, epochs)
print("Predicted Output: " ,(obj_ANN.forward_prop(x_test)))

Epoch 0: loss 0.5076988416510732
Epoch 1000: loss 0.05042850788530729
Epoch 2000: loss 0.0484809922359697
Epoch 3000: loss 0.045816822044778265
Epoch 4000: loss 0.01878949716744718
Epoch 5000: loss 0.011041254012898242
Epoch 6000: loss 0.011506194525613068
Epoch 7000: loss 0.01162505745119714
Epoch 8000: loss 0.011667985193914638
Epoch 9000: loss 0.011688166092461367
Predicted Output:  [[1.28988280e-03 9.99638402e-01 1.39428032e-04]
 [7.39283674e-06 2.08090004e-02 9.81010661e-01]
 [9.97871748e-01 1.75332271e-03 1.09619587e-08]
 [1.72915207e-03 9.99568492e-01 6.23051660e-05]
 [9.97801175e-01 1.95316169e-03 1.21681047e-08]
 [1.66387054e-03 9.99592926e-01 7.13735277e-05]
 [1.13391517e-05 3.45258190e-02 9.67026073e-01]
 [2.90932122e-03 9.99177476e-01 2.36359130e-05]
 [9.97801111e-01 1.93318825e-03 1.20080045e-08]
 [1.21469661e-03 9.99600899e-01 1.41803481e-04]
 [1.00898080e-03 9.99522501e-01 2.02677025e-04]
 [6.40293477e-06 1.86784978e-02 9.82384244e-01]
 [1.23807362e-03 9.99635684e-01 1.5

In [29]:
# Testing
print("\nPredictions after training:\n")
print("Input Features\tSL | SW | PL | PW\t    Predicted Class\t Actual Class")
for i in range(len(x_test)):
    predicted = obj_ANN.forward_prop(x_test[i])
    predicted_class = np.argmax(predicted)
    actual_class = y_test[i]
    print(f"\t\t{x_test[i]}\t\t    {predicted_class}\t\t     {actual_class}") # setosa: 1 , versicolor: 0 or virginica: 0


Predictions after training:

Input Features	SL | SW | PL | PW	    Predicted Class	 Actual Class
		[6.3 2.3 4.4 1.3]		    1		     [1]
		[6.4 2.7 5.3 1.9]		    2		     [2]
		[5.4 3.7 1.5 0.2]		    0		     [0]
		[6.1 3.  4.6 1.4]		    1		     [1]
		[5.  3.3 1.4 0.2]		    0		     [0]
		[5.  2.  3.5 1. ]		    1		     [1]
		[6.3 2.5 4.9 1.5]		    2		     [1]
		[5.8 2.7 4.1 1. ]		    1		     [1]
		[5.1 3.4 1.5 0.2]		    0		     [0]
		[5.7 2.8 4.5 1.3]		    1		     [1]
		[5.6 3.  4.5 1.5]		    1		     [1]
		[5.8 2.7 5.1 1.9]		    2		     [2]
		[5.5 2.3 4.  1.3]		    1		     [1]
		[4.9 3.  1.4 0.2]		    0		     [0]
		[5.1 3.8 1.5 0.3]		    0		     [0]
		[6.8 3.  5.5 2.1]		    2		     [2]
		[6.  3.4 4.5 1.6]		    1		     [1]
		[4.4 3.  1.3 0.2]		    0		     [0]
		[5.1 3.7 1.5 0.4]		    0		     [0]
		[5.  3.2 1.2 0.2]		    0		     [0]
		[7.1 3.  5.9 2.1]		    2		     [2]
		[6.4 2.8 5.6 2.2]		    2		     [2]
		[6.2 2.8 4.8 1.8]		    2		     [2]
		[4.8 3.4 1.9 0.2]		    0		     [0]
		[5.9 3.  4.2 

In [29]:
# Calculate accuracy
correct_count = 0
total = len(x_test)

for i in range(total):
    predicted = obj_ANN.forward_prop(x_test[i])
    predicted_class = np.argmax(predicted)
    if predicted_class == y_test[i]:
        correct_count += 1

accuracy = (correct_count / total) * 100
print(f"Accuracy: {accuracy:.2f}%")


Accuracy: 96.67%
